In [24]:
import pandas as pd
import json
from sklearn import metrics
from label_decoders import *

In [25]:
config = json.load(open('settings.json'))
train = pd.read_csv(config['train'])
test = pd.read_csv(config['test'])

In [26]:
# combine train and test
all_data = train.append(test,sort=False)
all_data.describe()

,Id,Product_Info_1,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,...,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response
count,79146.00000,79146.000000,79146.000000,79146.000000,79146.000000,79146.000000,79146.000000,79146.000000,79146.000000,79146.000000,...,79146.000000,79146.000000,79146.000000,79146.000000,79146.000000,79146.000000,79146.000000,79146.000000,79146.000000,59381.000000
mean,39573.50000,1.026015,24.392199,0.327810,2.006848,2.669143,1.043540,0.407898,0.706924,0.292580,...,0.057046,0.010310,0.045460,0.010550,0.007707,0.013658,0.008516,0.019508,0.054949,5.636837
std,22847.62654,0.159181,5.101213,0.281791,0.082470,0.743138,0.291821,0.198188,0.074306,0.089365,...,0.231933,0.101014,0.208313,0.102171,0.087453,0.116069,0.091888,0.138304,0.227882,2.456833
min,1.00000,1.000000,1.000000,0.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,19787.25000,1.000000,26.000000,0.076923,2.000000,3.000000,1.000000,0.238806,0.654545,0.225941,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
50%,39573.50000,1.000000,26.000000,0.230769,2.000000,3.000000,1.000000,0.417910,0.709091,0.288703,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000
75%,59359.75000,1.000000,26.000000,0.487179,2.000000,3.000000,1.000000,0.567164,0.763636,0.345188,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000
max,79146.00000,2.000000,38.000000,1.000000,3.000000,3.000000,3.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.000000


In [27]:
#create new variable for category feature
all_data['Product_Info_2_char'] = all_data['Product_Info_2'].str[0]
all_data['Product_Info_2_num'] = all_data['Product_Info_2'].str[1]
all_data['Product_Info_2'] = pd.factorize(all_data['Product_Info_2'])[0]
all_data['Product_Info_2_char'] = pd.factorize(all_data['Product_Info_2_char'])[0]
all_data['Product_Info_2_num'] = pd.factorize(all_data['Product_Info_2_num'])[0]

In [28]:
#new feature based on important features
all_data['BMI_Age'] = all_data['BMI'] * all_data['Ins_Age']

In [29]:
#handle various medical history features
med_keyword_columns = all_data.columns[all_data.columns.str.startswith('Medical_Keyword_')]
all_data['Med_Keywords_Count'] = all_data[med_keyword_columns].sum(axis=1)

In [30]:
all_data.apply(lambda x: sum(x.isnull()),1)

0         6
1         5
2         7
3         7
4         8
5         6
6         7
7         6
8         6
9         7
10        7
11        7
12        7
13        8
14        6
15        6
16        7
17        7
18        6
19        7
20        6
21        6
22        7
23        6
24        6
25        8
26        8
27        9
28        6
29        5
         ..
19735     8
19736     6
19737     6
19738     8
19739     7
19740     9
19741    10
19742     6
19743     8
19744     7
19745     7
19746     6
19747     6
19748    10
19749     7
19750     6
19751     7
19752     7
19753     7
19754     8
19755     7
19756     8
19757     8
19758     7
19759     9
19760     7
19761     7
19762     6
19763     9
19764     4
Length: 79146, dtype: int64

In [31]:
#null values
all_data['countna'] = all_data.apply(lambda x: sum(x.isnull()),1)
all_data.fillna(-1, inplace=True)

In [32]:
#Response type conversion
all_data['Response'] = all_data['Response'].astype(int)

In [33]:
# split train and test
train_ohd = all_data[all_data['Response']>0].copy()
test_ohd = all_data[all_data['Response']<1].copy()

In [34]:
#training and testing feature renaming/list
features=train_ohd.columns.tolist()
#features = [x.replace('=','_') for x in features]
features = [x.replace('_','i') for x in features]
train_ohd.columns = features

features_t=test_ohd.columns.tolist()
#features_t = [x.replace('=','i') for x in features_t]
features_t = [x.replace('_','i') for x in features_t]
test_ohd.columns = features_t

In [35]:
#remve Id and response from training features
features.remove("Id")
features.remove("Response")

In [36]:
train_ohd.dtypes

Id                       int64
ProductiInfoi1           int64
ProductiInfoi2           int64
ProductiInfoi3           int64
ProductiInfoi4         float64
ProductiInfoi5           int64
ProductiInfoi6           int64
ProductiInfoi7           int64
InsiAge                float64
Ht                     float64
Wt                     float64
BMI                    float64
EmploymentiInfoi1      float64
EmploymentiInfoi2        int64
EmploymentiInfoi3        int64
EmploymentiInfoi4      float64
EmploymentiInfoi5        int64
EmploymentiInfoi6      float64
InsuredInfoi1            int64
InsuredInfoi2            int64
InsuredInfoi3            int64
InsuredInfoi4            int64
InsuredInfoi5            int64
InsuredInfoi6            int64
InsuredInfoi7            int64
InsuranceiHistoryi1      int64
InsuranceiHistoryi2      int64
InsuranceiHistoryi3      int64
InsuranceiHistoryi4      int64
InsuranceiHistoryi5    float64
                        ...   
MedicaliKeywordi25       int64
Medicali

In [37]:
#
train_ohd['lr1'] = [0]*train_ohd.shape[0]
train_ohd['lr2'] = [0]*train_ohd.shape[0]
train_ohd['lr3'] = [0]*train_ohd.shape[0]
train_ohd['lr4'] = [0]*train_ohd.shape[0]
train_ohd['lr5'] = [0]*train_ohd.shape[0]
train_ohd['lr6'] = [0]*train_ohd.shape[0]
train_ohd['lr7'] = [0]*train_ohd.shape[0]
train_ohd['lr8'] = [0]*train_ohd.shape[0]
train_ohd['lr9'] = [0]*train_ohd.shape[0]
train_ohd['lr10'] = [0]*train_ohd.shape[0]
train_ohd['lr11'] = [0]*train_ohd.shape[0]
train_ohd['lr12'] = [0]*train_ohd.shape[0]
train_ohd['lr13'] = [0]*train_ohd.shape[0]

In [38]:
l = train_ohd.shape[0]
ind_list = [(list(range(0,l//10)), [x for x in range(0,l) if x not in list(range(0,l//10))]), 
            (list(range(l//10,l//10*2)), [x for x in range(0,l) if x not in list(range(l//10,l//10*2))]),
            (list(range(l//10*2,l//10*3)), [x for x in range(0,l) if x not in list(range(l//10*2,l//10*3))]),
            (list(range(l//10*3,l//10*4)), [x for x in range(0,l) if x not in list(range(l//10*3,l//10*4))]),
            (list(range(l//10*4,l//10*5)), [x for x in range(0,l) if x not in list(range(l//10*4,l//10*5))]),
            (list(range(l//10*5,l//10*6)), [x for x in range(0,l) if x not in list(range(l//10*5,l//10*6))]),
            (list(range(l//10*6,l//10*7)), [x for x in range(0,l) if x not in list(range(l//10*6,l//10*7))]),
            (list(range(l//10*7,l//10*8)), [x for x in range(0,l) if x not in list(range(l//10*7,l//10*8))]),
            (list(range(l//10*8,l//10*9)), [x for x in range(0,l) if x not in list(range(l//10*8,l//10*9))]),
            (list(range(l//10*9,l)), [x for x in range(0,l) if x not in list(range(l//10*9,l))])]

In [39]:
import csv
with open('indlist.csv','w') as f:
    writer = csv.writer(f)
    writer.writerows(ind_list)

In [21]:
ld = [labels_decoder1,labels_decoder2,labels_decoder3,labels_decoder4,labels_decoder5,labels_decoder6,labels_decoder7,
      labels_decoder8,labels_decoder9,labels_decoder10,labels_decoder11,labels_decoder12,labels_decoder13]

NameError: name 'labels_decoder1' is not defined